# Exceptions
 - Not all data is guaranteed to match our assumptions
 - When a section of python code cannot be executed an exception will be thrown
 - If this is an anticipated error we might catch that exception
 - We can also create our own Exceptions when values are not what they should be

The general syntax is:
```python
try:
    statement()
except ExceptionType:
    handling_statement()
```


<ins>Notes:</ins>
 - We are using indented blocks again
   - All statements in the try block will be executed one by one
   - If one fails, the interpreter checks whether an except block with a matching exception type exists and execute the code in there
   - We can have multiple except blocks with different exception types
 - We can leave out the exception type to catch all exceptions. This can easily lead to incorrect exception handling, when our assumption where the error comes from is wrong.

##  Exceptions often are used for known edge cases 


In [ ]:
def average(numbers):
    return sum(numbers) / len(numbers)

print(average([1,2,3,4]))
print(average([]))

# Modify the function to make an empty list return 0
def average_result(numbers):
    try:
        return sum(numbers) / len(numbers)
    except ZeroDivisionError:
        return 0

2.5


ZeroDivisionError: division by zero

<ins>Notes:</ins>
- Do __not__ write out the function again, modify the original function to look like the result function
- Demonstrate that you can catch the exception with a bare except
- Change the function to use the `ZeroDivisionError` explicitely, to avoid catching errors we do not mean to catch

## Reraising Meaningful exceptions

In [4]:
def parsed_max(values):
    max_value = None
    for value in values:
        parsed_value = float(value)
        if max_value is None or parsed_value > max_value:
            max_value = parsed_value
    return max_value

temperatures = ["23.5", "25.1", "22.8", "24.0"]
print(parsed_max(temperatures))

temperatures_with_error = ["23.5", "25.1", "2020-05-21", "24.0"]
print(parsed_max(temperatures_with_error))

# Modify function to list the index and content of the entry that caused the error
def parsed_max_result(values):
    max_value = None
    for i, value in enumerate(values):
        try:
            parsed_value = float(value)
            if max_value is None or parsed_value > max_value:
                max_value = parsed_value
        except ValueError as e:
            raise ValueError(f"Determining maximum: Could not convert '{value}' to float at index {i}") from e
    return max_value


25.1


ValueError: could not convert string to float: '2020-05-21'

<ins>Notes:</ins>
- Do __not__ write out the function again, modify the original function to look like the result function
- Using the from keyword we can add an exception to the error stack.
- This can be used to give additional information, that makes the error source in the input easily traceable

# Raising exceptions for invalid values

In [ ]:
def water_pressure(depth_m):
    g = 9.81  # acceleration due to gravity in m/s^2
    density = 1000  # density of water in kg/m^3
    pressure = density * g * depth_m
    return pressure

print(water_pressure(10))
print(water_pressure(-5))

# Modify function to check for negative depth
def water_pressure_result(depth_m):
    if depth_m < 0:
        raise ValueError("Water depth cannot be negative")
    g = 9.81  # acceleration due to gravity in m/s^2
    density = 1000  # density of water in kg/m^3
    pressure = density * g * depth_m
    return pressure


98100.0
-49050.0
98100.0


ValueError: Water depth cannot be negative

<ins>Notes:</ins>
- Do __not__ write out the function again, modify the original function to look like the result function
- It is good practice to have these checks early in a function. Nobody likes to be 10h into a script execution to then fail a validation

## The `finally` block will always be executed, either after code execution or after the failure

```python
try:
    statement()
except ExceptionType:
    # this block does not need to be here
    handle_error()
finally:
    always_execute()
```



In [ ]:
import time

def timed_calculation(n):
    start_time = time.time()
    # Some complex calculation
    if n < 0:
        raise ValueError("n must be positive")
    
    result = sum(i**2 for i in range(n))
    elapsed = time.time() - start_time
    print(f"Calculation took {elapsed:.6f} seconds")

    return result

# Test
timed_calculation(10000)
timed_calculation(-5)

# Modify function to always include the timing

def timed_calculation_result(n):
    start_time = time.time()
    try:
        # Some complex calculation
        if n < 0:
            raise ValueError("n must be positive")
        
        result = sum(i**2 for i in range(n))
        return result
    finally:
        # Always log the execution time
        elapsed = time.time() - start_time
        print(f"Calculation took {elapsed:.6f} seconds")

Calculation took 0.000854 seconds


ValueError: n must be positive

<ins>Notes:</ins>
- Do __not__ write out the function again, modify the original function to look like the result function
- Finally is often used in clean up operations.

## Best Practice:
If a value or input can occur that makes our script fail or produce a non-sensical result, we should try to (in that order)
 - Implement an alternative approach that works (possibly using except)
 - (Re-)raise an exception
   - If something has to fail, make it fail early. This might means non-sensical values but also values that would make a later statement fail.
   - Validation should be done within the programmatic unit that the validation is for
   - Provide meaningful exception messages, that let the user track the problem.

If you have trouble getting your script to run, users/collaborators will have that problem even more



## Have a play:


### Exercise 1: Grade Point Average Calculator

**Tasks:**
1. Modify the function to handle empty grade lists by returning 0.0
2. Add exception handling for non-numeric grades that provides a meaningful error message with the problematic grade and its position



In [ ]:
def calculate_gpa(grades):
    total_points = 0
    for grade in grades:
        total_points += grade
    return total_points / len(grades)

# Test cases
print(calculate_gpa([3.5, 4.0, 3.7, 3.9]))  # Should work
print(calculate_gpa([]))  # Will cause ZeroDivisionError
print(calculate_gpa([3.5, "A", 3.7]))  # Will cause TypeError

# Modify the function to handle unexpected values
def calculate_gpa(grades):
    try:
        total_points = 0
        for i, grade in enumerate(grades):
            try:
                total_points += grade
            except TypeError:
                raise ValueError(f"Invalid grade '{grade}' at position {i}. Expected numeric value.") from None
        return total_points / len(grades)
    except ZeroDivisionError:
        return 0.0



### Exercise 2: Research Data Processor
**Tasks:**
1. Add validation to ensure control samples have exactly 3 measurements, raising a `ValueError` with sample ID if not
2. Handle the case where experimental samples have no measurements by returning `None`
3. Add a try-except block that catches any unexpected errors and re-raises them with additional context about which sample failed



In [ ]:
def process_sample_data(sample_id, measurements):
    if sample_id.startswith("CTRL"):
        # Control samples should have exactly 3 measurements
        baseline = measurements[2]  # Use third measurement as baseline
    else:
        # Experimental samples
        baseline = sum(measurements) / len(measurements)
    
    return baseline * 1.5  # Apply correction factor

# Test cases
print(process_sample_data("EXP001", [12.5, 14.2, 13.8]))  # Should work
print(process_sample_data("CTRL001", [10.1, 11.5]))  # IndexError - only 2 measurements
print(process_sample_data("EXP002", []))  # ZeroDivisionError

# Modify the function to handle the test cases
def process_sample_data(sample_id, measurements):
    try:
        if sample_id.startswith("CTRL"):
            # Control samples should have exactly 3 measurements
            if len(measurements) != 3:
                raise ValueError(f"Control sample {sample_id} must have exactly 3 measurements, got {len(measurements)}")
            baseline = measurements[2]  # Use third measurement as baseline
        else:
            # Experimental samples
            if len(measurements) == 0:
                return None
            baseline = sum(measurements) / len(measurements)
        
        return baseline * 1.5  # Apply correction factor
    except ValueError:
        raise  # Re-raise our custom validation errors
    except Exception as e:
        raise RuntimeError(f"Failed to process sample {sample_id}: {str(e)}") from e



```python

```



---

## Notes:
- Do **not** write out the function again, modify the original function to look like the result function
- Test your modifications with the provided test cases
- Use specific exception types where possible to avoid catching unintended errors